
This notebook documents a minimal working example of the "ValueError: Buffer source array is read-only" that I get when setting `n_jobs > 1` in `GridSearchCV`.  

The use case is the following: I have a dataframe with a mix of string and numeric columns. I build a Pipeline that first encodes the data properly and then passes the encoded data into a classifier. I encode the data using a custom class called `DataFrame_Encoder` to specify which columns should be one-hot encoded with DictVectorizer and which columns should be kept as is. I then pass that Pipeline into GridSearchCV to optimize over the hyperparameters of the classifier. 

This snippet crashes when `n_jobs > 1` in `GridSearchCV` and the dataset is large (breaks somewhere between 100K and 200K rows but I haven't found the exact breaking point. 

The error occurs whenever a Dataframe that contains a column of dtype Object is passed to `GridSearchCV.fit`. I initially believed that the error was caused by my custom `DataFrame_Encoder` class but I no longer that is the case. The code breaks prior to ever calling the `fit` or `transform` methods of `DataFrame_Encoder`. I think the error is happening as soon as `GridSearchCV.fit` is called. Perhaps there's a check that works successfully for non-Object columns but doesn't work for Object columns. 

- Example 1 shows the MWE that results in the ValueError
- Example 2&3 show that the error is not caused by my custom DataFrame_Encoder class.
- Example 4 shows that things work if you drop the Object column from the Dataframe. 
- Example 5 shows that things work if you decrease the number of rows in the dataset.

## Related issues

This bug seems to be related to the following issues: 

- https://github.com/pandas-dev/pandas/issues/9928#issuecomment-97038944
- https://github.com/scikit-learn/scikit-learn/issues/4772
- https://github.com/scikit-learn/scikit-learn/pull/4775

It seems like it was supposed to be fixed in a recent version, so perhaps this is separate thing. 

In [1]:
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer

import numpy as np


In [2]:
class DataFrame_Encoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical_cols_=None,numeric_cols_=None):
        print("__init__ called")
        self.categorical_cols_ = categorical_cols_
        self.numeric_cols_ = numeric_cols_
    
    def fit(self, df, y=None):
        print("Fit called")
        ### df should be a dataframe that is a mix of categorical and numeric columns
        self.vec_ = DictVectorizer(sparse=False)
        temp_data = df[self.categorical_cols_].astype(str)
        self.vec_.fit(temp_data.to_dict('records'))
        self.feature_names_ = list(self.numeric_cols_) + list(self.vec_.feature_names_)
        return self

    def transform(self, df):
        ### df should be a dataframe that is a mix of categorical and numeric columns
        print("Transform called")
        temp_data = df[self.categorical_cols_].astype(str)
        categorical_data = self.vec_.transform(temp_data.to_dict('records'))
        categorical_df = pd.DataFrame(categorical_data, columns=self.vec_.feature_names_, index=df.index)
        new_data = pd.concat([df[self.numeric_cols_], categorical_df],axis=1)
        return new_data


# Example 1 : Fails 

In [3]:
x,y = make_classification(n_samples=200000,n_features=5)

numeric_features = ['x1','x2','x3','x4','x5']
string_features = ['category']

df = pd.DataFrame(data=x,columns=numeric_features)
df['category'] = 'a'

base_clf = RandomForestClassifier(n_jobs=4)
param_grid = {'clf__n_estimators':[10,100]}

pipeline = Pipeline([
        ('feature_encoder',DataFrame_Encoder()),
        ('clf',base_clf)
])
pipeline.set_params(feature_encoder__categorical_cols_=string_features, feature_encoder__numeric_cols_=numeric_features)

clf = GridSearchCV(pipeline, param_grid,cv=5,n_jobs=2,verbose=1)

clf.fit(df,y)


__init__ called
Fitting 5 folds for each of 2 candidates, totalling 10 fits
__init__ called
__init__ called

/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)



__init__ called
__init__ called
__init__ called


/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Use

#  Example 2: Fails

First we attempt to get rid of the string columns (by setting the list `string_features` to be empty) and see if that fixes anything. It doesn't. Furthermore, a glance at the output shows that `fit` and `transform` are never called. 



In [4]:
x,y = make_classification(n_samples=200000,n_features=5)

numeric_features = ['x1','x2','x3','x4','x5']
string_features = []


df = pd.DataFrame(data=x,columns=numeric_features)
df['category'] = 'a'

base_clf = RandomForestClassifier(n_jobs=4)
param_grid = {'clf__n_estimators':[10,100]}

pipeline = Pipeline([
        ('feature_encoder',DataFrame_Encoder()),
        ('clf',base_clf)
])
pipeline.set_params(feature_encoder__categorical_cols_=string_features, feature_encoder__numeric_cols_=numeric_features)

clf = GridSearchCV(pipeline, param_grid,cv=5,n_jobs=2,verbose=1)

clf.fit(df,y)


__init__ called
Fitting 5 folds for each of 2 candidates, totalling 10 fits
__init__ called
__init__ called
__init__ called
__init__ called
__init__ called


/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Use

# Example 3: Fails

We now use a different encoder that simply drops any Object columns from a Dataframe. Even with this encoder, it fails. 

In [5]:
class DropString_Encoder(BaseEstimator, TransformerMixin):
    
    def fit(self,df,y=None):
        return self
    
    def transform(self,df):
        non_string_cols = df.select_dtypes(exclude=[object]).columns.values
        return df[non_string_cols]


    
x,y = make_classification(n_samples=200000,n_features=5)

numeric_features = ['x1','x2','x3','x4','x5']
string_features = ['category']

df = pd.DataFrame(data=x,columns=numeric_features)
df['category'] = 'a'

base_clf = RandomForestClassifier(n_jobs=4)
param_grid = {'clf__n_estimators':[10,100]}

pipeline = Pipeline([
        ('feature_encoder',DropString_Encoder()),
        ('clf',base_clf)
])

clf = GridSearchCV(pipeline, param_grid,cv=5,n_jobs=2,verbose=1)

clf.fit(df,y)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/site-packages/sklearn/utils/__init__.py:104: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/gstoddard/anaconda/envs/standard_py3_env/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Use

# Example 4: Works
 
In line 20, we drop the one Object column and only keep the numeric ones. Once we do that, things work fine. 

In [7]:
x,y = make_classification(n_samples=200000,n_features=5)

numeric_features = ['x1','x2','x3','x4','x5']
string_features = []

df = pd.DataFrame(data=x,columns=numeric_features)
df['category'] = 'a'

base_clf = RandomForestClassifier(n_jobs=4)
param_grid = {'clf__n_estimators':[10]}

pipeline = Pipeline([
        ('feature_encoder',DataFrame_Encoder()),
        ('clf',base_clf)
])
pipeline.set_params(feature_encoder__categorical_cols_=string_features, feature_encoder__numeric_cols_=numeric_features)

clf = GridSearchCV(pipeline, param_grid,cv=2,n_jobs=2,verbose=1)

temp_df = df[numeric_features]

clf.fit(temp_df,y)


__init__ called
Fitting 2 folds for each of 1 candidates, totalling 2 fits
__init__ called
__init__ called
__init__ called
Fit called
Fit called
Transform called
Transform called
Transform called
Transform called
Transform called
Transform called


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.9s finished


__init__ called
Fit called
Transform called


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(steps=[('feature_encoder', DataFrame_Encoder(categorical_cols_=[],
         numeric_cols_=['x1', 'x2', 'x3', 'x4', 'x5'])), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impuri...imators=10, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'clf__n_estimators': [10]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=1)

# Example 5: Works
 
This is the full example but we simply decrease the size of the dataset and everything works fine. 

In [8]:
x,y = make_classification(n_samples=2000,n_features=5)

numeric_features = ['x1','x2','x3','x4','x5']
string_features = ['category']

df = pd.DataFrame(data=x,columns=numeric_features)
df['category'] = 'a'

base_clf = RandomForestClassifier(n_jobs=4)
param_grid = {'clf__n_estimators':[10]}

pipeline = Pipeline([
        ('feature_encoder',DataFrame_Encoder()),
        ('clf',base_clf)
])
pipeline.set_params(feature_encoder__categorical_cols_=string_features, feature_encoder__numeric_cols_=numeric_features)

clf = GridSearchCV(pipeline, param_grid,cv=2,n_jobs=2,verbose=1)


clf.fit(df,y)


__init__ called
Fitting 2 folds for each of 1 candidates, totalling 2 fits
__init__ called
Fit called
Fit called
Transform called
Transform called
__init__ called
__init__ called
Transform called
Transform called
Transform called
Transform called


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.4s finished


__init__ called
Fit called
Transform called


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(steps=[('feature_encoder', DataFrame_Encoder(categorical_cols_=['category'],
         numeric_cols_=['x1', 'x2', 'x3', 'x4', 'x5'])), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            ...imators=10, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'clf__n_estimators': [10]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=1)

# Versions

In [9]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)

Darwin-15.6.0-x86_64-i386-64bit
Python 3.6.1 |Continuum Analytics, Inc.| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
NumPy 1.13.1
SciPy 0.19.1
Scikit-Learn 0.18.2
